# Домашняя работа по теме "Transfer learning"

__Задание__

* взять данные из https://www.kaggle.com/datasets/mrapplexz/bashim-quotes обучить модель GPT для генерации своих цитат
* взять новостные данные из https://github.com/natasha/corus load_lenta2 нам понадобиться сам текст и заголовок обучить модель T5/ или GPT для генерации заголовков для статей


In [1]:
from google.colab import drive

In [2]:
drive.mount('/drive')

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [2]:
!pip install datasets transformers==4.28.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.6 MB/s eta 0:00:00


In [3]:
import numpy as np
import pandas as pd

In [5]:
import logging
from transformers.trainer import logger as noisy_logger
noisy_logger.setLevel(logging.WARNING)
import warnings
warnings.filterwarnings("ignore")

In [6]:
model_name = "bankholdup/rugpt3_song_writer"

In [7]:
df = pd.read_json('/drive/MyDrive/GB/dataset.jsonl', lines=True).set_index('id')

In [8]:
df.head(5)

,date,rating,text
id,,,
1,2004-08-30 11:24:00+00:00,22010.0,"<Ares> ppdv, все юниксы очень дружелюбны.. они..."
2,2004-08-30 11:25:00+00:00,25105.0,<томатик_рад> а ты не чувствуешь красоту мира?...
3,2004-08-30 11:27:00+00:00,7192.0,"<Дор> ""мышка, почему у тебя такие большие глаз..."
4,2004-08-30 11:28:00+00:00,29169.0,"<PPDV[os2]> ""Мальчики, вы что больные, бегать ..."
5,2004-08-30 11:26:00+00:00,7140.0,<Ohtori_Akio> мы - как разработчики - живём с ...


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 81497 entries, 1 to 463648
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype              
---  ------  --------------  -----              
 0   date    81497 non-null  datetime64[ns, UTC]
 1   rating  80642 non-null  float64            
 2   text    81497 non-null  object             
dtypes: datetime64[ns, UTC](1), float64(1), object(1)
memory usage: 2.5+ MB


## Предобработка текста

In [10]:
import re
def clear_text(text):
    clr_text = re.sub(r"<.*?>", " ", text).lower()
    clr_text = summary = re.sub(r"\s", " ", clr_text)
    return clr_text

In [11]:
df["clear_text"] = df["text"].apply(lambda x: clear_text(x))
df.head()

,date,rating,text,clear_text
id,,,,
1,2004-08-30 11:24:00+00:00,22010.0,"<Ares> ppdv, все юниксы очень дружелюбны.. они...","ppdv, все юниксы очень дружелюбны.. они прос..."
2,2004-08-30 11:25:00+00:00,25105.0,<томатик_рад> а ты не чувствуешь красоту мира?...,а ты не чувствуешь красоту мира? честно го...
3,2004-08-30 11:27:00+00:00,7192.0,"<Дор> ""мышка, почему у тебя такие большие глаз...","""мышка, почему у тебя такие большие глаза?"" ..."
4,2004-08-30 11:28:00+00:00,29169.0,"<PPDV[os2]> ""Мальчики, вы что больные, бегать ...","""мальчики, вы что больные, бегать в палату к..."
5,2004-08-30 11:26:00+00:00,7140.0,<Ohtori_Akio> мы - как разработчики - живём с ...,мы - как разработчики - живём с субейзом под...


In [12]:
data = df.loc[:, 'clear_text']
data

id
1           ppdv, все юниксы очень дружелюбны.. они прос...
2           а ты не чувствуешь красоту мира?   честно го...
3           "мышка, почему у тебя такие большие глаза?" ...
4           "мальчики, вы что больные, бегать в палату к...
5           мы - как разработчики - живём с субейзом под...
                                ...                        
463644    xxx: угадайте не гугля, что такое жопиздан! yy...
463645    xxx: посетила шальная мысль заняться собой, жи...
463646    #всебожьяроса xxx: судьба айтишников вообще не...
463647    прибывшие на место правоохранители установили,...
463648    ххх: вот ребята из английской фирмы tvr (давны...
Name: clear_text, Length: 81497, dtype: object

## Разбивка на обучающую и тестовую выборки

In [13]:
from sklearn.model_selection import train_test_split

def build_text_files(data_json, dest_path):
    f = open(dest_path, 'w')
    data = ''
    for texts in data_json:
        summary = str(texts).strip()
        data += summary + "  "
    f.write(data)

In [14]:
train, test = train_test_split(data, test_size=0.2)

build_text_files(train,'train_dataset.txt')
build_text_files(test,'test_dataset.txt')

In [15]:
print("Train dataset length: "+ str(len(train)))
print("Test dataset length: "+ str(len(test)))

Train dataset length: 65197
Test dataset length: 16300


Загрузка токенайзера из предобученной модели

In [16]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)

train_path = 'train_dataset.txt'
test_path = 'test_dataset.txt'

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [17]:
from transformers import TextDataset, DataCollatorForLanguageModeling

def load_dataset(train_path, test_path, tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)

    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset, test_dataset, data_collator

train_dataset, test_dataset, data_collator = load_dataset(train_path, test_path, tokenizer)

## Загрузка модели

In [18]:
from transformers import Trainer, TrainingArguments, AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(model_name)

### Настройка параметров модели

In [19]:
training_args = TrainingArguments(

    "phrase",
    evaluation_strategy = "epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=2,
    learning_rate=1e-5,
    weight_decay=0.01,
    save_strategy='no',
    report_to='none',

    )

In [20]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

## Обучение модели

In [21]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,4.128200,3.993395
2,3.993100,3.961312


TrainOutput(global_step=16968, training_loss=4.1138158543724765, metrics={'train_runtime': 3327.5189, 'train_samples_per_second': 20.396, 'train_steps_per_second': 5.099, 'total_flos': 4433341906944000.0, 'train_loss': 4.1138158543724765, 'epoch': 2.0})

## Генерация текста

In [22]:
def generate_text(prefix):
    tokens = tokenizer(prefix, return_tensors='pt').to('cuda')
    size = tokens['input_ids'].shape[1]

    output = model.generate(
        **tokens,
        #end_token=end_token_id,
        do_sample=False,
        max_length=size+50,
        early_stopping=True,
        length_penalty=2.0,
        repetition_penalty=8.,
        temperature=0.5,
        num_beams=3,
        no_repeat_ngram_size=5
    )

    decoded = tokenizer.decode(output[0])
    result = decoded[len(prefix):]
    return prefix + result

In [23]:
print(generate_text("Здравствуй!"))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Здравствуй!  xxx: у нас в школе был препод по истории. он рассказывал, что когда-то давным-давно была война между римлянами и франками за контрольную работу на тему "что такое империя". так вот, после того как


In [24]:
print(generate_text("Сегодня"))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Сегодня, когда я был маленьким, у нас в доме было две комнаты: одна для взрослых, а вторая для детей. и вот как-то раз мы с мамой пришли к нам гости - дети играли на пианино...  xxx: если бы


In [25]:
print(generate_text("В каком году"))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


В каком году в россии начнут вводить налоги на алкоголь?  xxx: у меня есть знакомый, который занимается продажей алкоголя. и вот однажды он пришел к нам домой с бутылкой водки "вся жизнь заебись". я его спросил - а


## Вторая часть задания

In [4]:
!pip install razdel networkx pymorphy2 nltk rouge==0.3.1

  Using cached razdel-0.5.0-py3-none-any.whl (21 kB)
  Using cached pymorphy2-0.9.1-py3-none-any.whl (55 kB)
  Using cached rouge-0.3.1-py3-none-any.whl (7.4 kB)
  Using cached DAWG_Python-0.7.2-py2.py3-none-any.whl (11 kB)
  Using cached pymorphy2_dicts_ru-2.4.417127.4579844-py2.py3-none-any.whl (8.2 MB)
  Using cached docopt-0.6.2-py2.py3-none-any.whl


In [5]:
from datasets import load_dataset

dataset_train = load_dataset('IlyaGusev/gazeta', revision="v1.0", split= 'train[:10%]')
dataset_test = load_dataset('IlyaGusev/gazeta', revision="v1.0", split= 'test[:10%]')

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/52400 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5770 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5265 [00:00<?, ? examples/s]

In [6]:
dataset_train

Dataset({
    features: ['text', 'summary', 'title', 'date', 'url'],
    num_rows: 5240
})

In [7]:
dataset_test

Dataset({
    features: ['text', 'summary', 'title', 'date', 'url'],
    num_rows: 577
})

In [8]:
dataset_test['summary'][0]

'В NASA назвали четыре миссии в дальний космос, которые в этом десятилетии могут быть запущены американцами. Среди них — две миссии по изучению Венеры, полет к спутнику Юпитера и экспедиция к Тритону, спутнику Нептуна.'

In [9]:
dataset_test['title'][0]

'Венера, Ио или Тритон: куда полетит NASA'

In [10]:
model_name = "IlyaGusev/rut5_base_sum_gazeta"

In [11]:
def len_tok(text):
    return len(text.split())

In [12]:
max_len_sum, max_len_tl = max(map(len_tok, dataset_train['summary'])), max(map(len_tok, dataset_train['title']))
max_len_sum, max_len_tl

(75, 18)

In [13]:
max_len_sum, max_len_tl = 60, 15

In [14]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    tokenized_input = tokenizer(batch['summary'], padding='max_length', truncation=True, max_length=max_len_sum)
    tokenized_label = tokenizer(batch['title'], padding='max_length', truncation=True, max_length=max_len_tl)

    tokenized_input['labels'] = tokenized_label['input_ids']

    return tokenized_input

dataset_train = dataset_train.map(tokenize, batched=True, batch_size=8)
dataset_test = dataset_test.map(tokenize, batched=True, batch_size=8)

dataset_train.set_format('numpy', columns=['input_ids', 'attention_mask', 'labels'])
dataset_test.set_format('numpy', columns=['input_ids', 'attention_mask', 'labels'])

Map:   0%|          | 0/5240 [00:00<?, ? examples/s]

Map:   0%|          | 0/577 [00:00<?, ? examples/s]

In [15]:
dataset_train.save_to_disk('train_data')
dataset_test.save_to_disk('test_data')

Saving the dataset (0/1 shards):   0%|          | 0/5240 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/577 [00:00<?, ? examples/s]

In [16]:
from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments


model = T5ForConditionalGeneration.from_pretrained(model_name)

In [19]:
output_dir = 'output'

training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_accumulation_steps=1, # Number of eval steps to keep in GPU (the higher, the mor vRAM used)
    prediction_loss_only=True, # If I need co compute only loss and not other metrics, setting this to true will use less RAM
    learning_rate=0.00001,
    evaluation_strategy='steps', # Run evaluation every eval_steps
    save_steps=1000, # How often to save a checkpoint
    save_total_limit=1, # Number of maximum checkpoints to save
    remove_unused_columns=True, # Removes useless columns from the dataset
    run_name='run_gazeta', # Wandb run name
    logging_steps=500, # How often to log loss to wandb
    eval_steps=500, # How often to run evaluation on the val_set
    logging_first_step=False, # Whether to log also the very first training step to wandb
    load_best_model_at_end=True, # Whether to load the best model found at each evaluation.
    metric_for_best_model="loss", # Use loss to evaluate best model.
    greater_is_better=False # Best model is the one with the lowest loss, not highest.
)

In [20]:
%%time

# Обучение. У нас 4 эпохи.
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_test
)

trainer.train()

Step,Training Loss,Validation Loss
500,6.420500,3.264914
1000,4.529300,3.167768
1500,2.738900,3.057399
2000,2.625700,3.005496
2500,2.573900,2.987539


CPU times: user 9min 45s, sys: 44 s, total: 10min 28s
Wall time: 11min 10s


TrainOutput(global_step=2620, training_loss=3.7224584098990636, metrics={'train_runtime': 669.9992, 'train_samples_per_second': 31.284, 'train_steps_per_second': 3.91, 'total_flos': 1669603442688000.0, 'train_loss': 3.7224584098990636, 'epoch': 4.0})

In [21]:
trainer.save_model(output_dir + '/model')

In [32]:
INX = 90
print("SUMMARY: | {}".format(dataset_test['summary'][INX]))
print("TITLE: | {}".format(dataset_test['title'][INX]))

SUMMARY: | Глава МИД Белоруссии Владимир Макей выразил уверенность, что программа интеграции не несет опасности для суверенитета республики. Дипломат считает, что в Белоруссии сейчас живет общество, выросшее в независимой стране, и, соответственно, не готовое жертвовать суверенитетом страны. Политик указал на препятствия для интеграции, а также заявил о недопустимости нивелирования ранее принятых договоренностей.
TITLE: | «Пожертвовать независимостью»: Минск рассказал об интеграции с РФ


In [24]:
device = "cuda"

In [33]:
import torch

input_text = dataset_test['summary'][INX]

with torch.no_grad():
    tokenized_text = tokenizer(input_text, truncation=True, padding=True, return_tensors='pt')

    source_ids = tokenized_text['input_ids'].to(device, dtype = torch.long)
    source_mask = tokenized_text['attention_mask'].to(device, dtype = torch.long)

    generated_ids = model.generate(
        input_ids = source_ids,
        attention_mask = source_mask,
        max_length=512,
        num_beams=7,
        temperature = 1.3,
        repetition_penalty=1,
        length_penalty=1,
        early_stopping=True,
        no_repeat_ngram_size=2  # количество повторов n-грамм > 2 запрещено.
    )

    # Параметры подбираются эксперементально

    pred = tokenizer.decode(generated_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)

print("\noutput:\n" + pred)


output:
Белоруссия не готова к интеграции


In [34]:
INX = 30
print("SUMMARY: | {}".format(dataset_test['summary'][INX]))
print("TITLE: | {}".format(dataset_test['title'][INX]))

input_text = dataset_test['summary'][INX]

with torch.no_grad():
    tokenized_text = tokenizer(input_text, truncation=True, padding=True, return_tensors='pt')

    source_ids = tokenized_text['input_ids'].to(device, dtype = torch.long)
    source_mask = tokenized_text['attention_mask'].to(device, dtype = torch.long)

    generated_ids = model.generate(
        input_ids = source_ids,
        attention_mask = source_mask,
        max_length=512,
        num_beams=7,
        temperature = 1.3,
        repetition_penalty=1,
        length_penalty=1,
        early_stopping=True,
        no_repeat_ngram_size=2  # количество повторов n-грамм > 2 запрещено.
    )

    # Параметры подбираются эксперементально
    pred = tokenizer.decode(generated_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)

print("\noutput:\n" + pred)

SUMMARY: | Почти 80% россиян считают важными предложенные президентом поправки к Конституции. По опросу ВЦИОМ, наибольшей поддержкой пользуются социальные инициативы главы государства — их одобрили более 90% респондентов. Кроме того, россияне положительно оценивают ужесточение требований к кандидатам в президенты, а также усиление роли парламента.
TITLE: | Изменение Конституции: россияне оценили предложение президента

output:
Почти 80% россиян считают важными поправки к Конституции
